In [1]:
!pip install glove_python

     |████████████████████████████████| 266kB 3.4MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700261 sha256=3cb72a5574d07a6eda9c4cc2e9c850399e25a7be24d4668c38182a44bc192108
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [2]:
# google drive 연동
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
import os
os.getcwd()
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [4]:
from gensim.models import Word2Vec, KeyedVectors
import csv
import re

def open_csv():
    # csv파일을 연다!
    f = open('IMDB Dataset.csv', 'r', encoding='utf-8')
    csvreader = csv.reader(f)
    
    doc_list = []

    next(csvreader)
    for f in csvreader:
        line = re.compile("[^\w]").sub(' ', f[0].lower())
        doc_list.append(line.split())

    return doc_list

doc_list = open_csv()

In [5]:
from glove import Corpus, Glove 
# corpus : 단어, 텍스트를 저장하는 객체 (like list)

corpus = Corpus()
corpus.fit(doc_list, window=3)



# 임베딩을 최적화해 나갈 때 경사하강법을 사용함
glove = Glove(no_components=50, learning_rate=0.05)
  # no_components : output vector size

# training-------------------------------------------------------
glove.fit(corpus.matrix, epochs=10, no_threads=4, verbose=True)
  # no_threads : workers
  # verbose : 훈련 내용 출력

Performing 10 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9


In [6]:
# 유사도 검색---------------------------------------------------
glove.add_dictionary(corpus.dictionary) # 행렬의 index에 해당하는 정보를 단어 이름으로 검색할 수 있게 함

In [7]:
glove_result1 = glove.most_similar('dog')
print(glove_result1)

[('baby', 0.9130754558123968), ('business', 0.9062528806873447), ('horse', 0.8964228286429897), ('victim', 0.8939840635263233)]


실제로는 most_similar를 많이 사용하지는 않음. 이를 통해 단어 임베딩을 하는 것이 목적임.

---

# 케라스 토크나이저

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences # padding


token = Tokenizer()
text = "This tokenizer is good"
token.fit_on_texts([text])

# 단어 집합---------------------------------
print(token.word_index)
# 단어 빈도---------------------------------
print(token.word_counts)
# 인덱스 부여, 정수 인코딩------------------
text_2 = "This tokenizer is not good good"
int_encoding = token.texts_to_sequences([text_2]) # 만들어진 tokenizer를 이용해서 index를 지정
print(int_encoding) # [1, 2, 3, 4, 4] : not은 이전 텍스트에 없었기 때문에 index 출력X

# padding-----------------------------------
# 샘플의 길이를 맞출 때 사용
# 없는 부분을 0으로 표시(pre면 앞에 0, post면 뒤에 0)
padding = pad_sequences([[1, 2], [3, 4], [5, 6]], maxlen=3, padding='pre')
print(padding)

# 행렬화------------------------------------
text_list= ['안녕하세요 감사해요 잘있어요 다시 만나요',
    '안녕하세요 오늘도 좋은 하루 보내세요',
    '안녕하세요 안녕하세요 안녕하세요 오늘도 날씨가 좋네요']
token.fit_on_texts(text_list)
matrix = token.texts_to_matrix(text_list, mode='count')
       # mode : count는 dtm, binary는 존재 여부 표시, freq 빈도율, tfidf는 tfidf 행렬
print(matrix)

{'this': 1, 'tokenizer': 2, 'is': 3, 'good': 4}
OrderedDict([('this', 1), ('tokenizer', 1), ('is', 1), ('good', 1)])
[[1, 2, 3, 4, 4]]
[[0 1 2]
 [0 3 4]
 [0 5 6]]
[[0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.]
 [0. 3. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]]


In [16]:
print(token.word_index)

{'안녕하세요': 1, '오늘도': 2, 'this': 3, 'tokenizer': 4, 'is': 5, 'good': 6, '감사해요': 7, '잘있어요': 8, '다시': 9, '만나요': 10, '좋은': 11, '하루': 12, '보내세요': 13, '날씨가': 14, '좋네요': 15}
